In [ ]:
# import文
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import optuna
import mlflow

import modules as mod

In [ ]:
# SQLiteのDBファイル作成先の指定
DB_PATH= '[バックエンド用DBファイルを作成するパス] 例: "/Users/user-name/Desktop/Backend/Tracking/mlruns.db" '
# アーティファクト保存先の指定
ARTIFACT_LOCATION = '[Artifactストレージに指定したいパス] 例: "/Users/user-name/Desktop/Backend/Artifact" '
# Experiment名の指定
EXPERIMENT_NAME = '[作成/指定したいエクスペリメント名] 例: "optuna-mlflow '

In [ ]:
# バックエンドの準備
tracking_uri = mod.backend(db_path=DB_PATH)

In [ ]:
# Experimentの生成
experiment_id = mod.create_experiment(experiment_name=EXPERIMENT_NAME, artifact_loc=ARTIFACT_LOCATION)

In [ ]:
# データの準備
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# データの正規化
x_train, x_test = x_train / 255.0, x_test / 255.0

# ラベル情報の辞書を作成
label_description = {
    0:"T-shirt/top",
    1:"Trouser",
    2:"Pullover",
    3:"Dress",
    4:"Coat",
    5:"Sandal",
    6:"Shirt",
    7:"Sneaker",
    8:"Bag",
    9:"Ankle boot"
}

In [ ]:
# データの中身を確認

# subplotsで描画を設定：引数で描画領域の行数/列数, x/y軸の統一を指定
fix, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(10):
    # 各ファッションデータのうち最初に出てきたデータをピックアップする
    img = x_train[y_train == i][0]
    ax[i].imshow(img, cmap='Greys')
    ax[i].set_title(label_description[i]) 

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# 多層ニューラルネットワークモデル作成用の関数
def create_model(trial):
    
    # ドロップアウト率の探索範囲を設定
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5, log=False)
    
    # AdamWの学習率の探索範囲を設定
    optimeizer = tf.keras.optimizers.AdamW(
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    )

    # 多層FNNの構築
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28))
    ])
    for i in range(1,5):
        model.add(tf.keras.layers.Dense(units=512, name=f'hidden_{i}', activation='relu'))
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(tf.keras.layers.Dense(units=10, name='output', activation='softmax'))

    # 最適化手法、損失関数、評価指標の設定
    model.compile(optimizer=optimeizer,
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ["accuracy"])
    
    return model

In [ ]:
# optunaに渡すためのmlflow-callbackの準備
mlflc = mod.mlflow_callback(tracking_uri=tracking_uri, experiment_id=experiment_id)

In [ ]:
# ハイパーパラメータの指定（最適化対象外）
tf.random.set_seed(1)
EPOCHS = 50
BATCH_SIZE = 32

validation_split = 0.2
step_per_epoch = np.ceil(x_train.shape[0] * (1-validation_split) / BATCH_SIZE)    
validation_steps = np.ceil(x_train.shape[0] * validation_split / BATCH_SIZE)   

In [ ]:
# optunaにより最小化したい目的関数を準備
@mlflc.track_in_mlflow()
def objective(trial):
     
    # モデルの生成
    model = create_model(trial)
    
    # 過学習対策に EarlyStopping コールバックを設定。val_lossの値が３エポックに渡って改善されなかった場合に学習を中止する
    # 効率化のため TFKerasPruning コールバックを設定。精度が出る見込みが薄いハイパーパラメータの組み合わせについては早々に切り捨てる
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        optuna.integration.TFKerasPruningCallback(trial, monitor='val_accuracy'),
    ]
    
    # モデルの訓練
    history = model.fit(x_train, y_train,
                        validation_split=validation_split,
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCHS, 
                        steps_per_epoch=step_per_epoch, 
                        validation_steps=validation_steps,
                        callbacks=callbacks,
                        verbose=0)
    
    # 現在のトライアルのハイパーパラメータ（trial.params）をrunに記録
    mlflow.log_params(trial.params)
    # 現在のトライアルの評価結果をrunに記録
    mlflow.log_metrics({'val_accuracy' : history.history['val_accuracy'][-1]})

    # 最後の val_accuracy を出力
    return history.history['val_accuracy'][-1]

In [ ]:
# 最適化を実行
if __name__ == "__main__":
    # studyの作成。'枝刈り'の方法としてはMedianPrunerを設定
    study = optuna.create_study(
        direction='maximize', pruner=optuna.pruners.MedianPruner(n_startup_trials=2)
        )
    
    # 最適化の実施
    study.optimize(objective, n_trials=100, callbacks=[mlflc])

    # 途中で枝刈りされたトライアルの数と、最後まで完了したトライアルの数を取得
    pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
    
    # トライアル回数の確認
    print("Study statistics: ")
    print(f" Number of finished trials : {len(study.trials)} ")
    print(f" Number of pruned trials : {len(pruned_trials)} ")
    print(f" Number of complete trials : {len(complete_trials)} ")

    # 最良のトライアルの確認
    print(" Best trial : ")
    best_trial = study.best_trial

    print(f" Value : {best_trial.value} ")

    print(" Params : ")
    for key, value in best_trial.params.items():
        print(f" {key} : {value} ")

In [ ]:
# 最も正解率の高かったハイパーパラメータの組み合わせを用いてモデルを生成し訓練を実施する
best_model = create_model(best_trial)

# 過学習対策で EarlyStopping を設定
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
]

# モデルの訓練
history = best_model.fit(x_train, y_train,
                         validation_split=validation_split,
                         batch_size=BATCH_SIZE, 
                         epochs=EPOCHS, 
                         steps_per_epoch=step_per_epoch, 
                         validation_steps=validation_steps,
                         callbacks=callbacks,
                         verbose=1)

# モデルをMLflowで管理
with mlflow.start_run(experiment_id=experiment_id):
    mlflow.tensorflow.log_model(best_model, artifact_path="optuna-fashion_mnist")

In [ ]:
# テストデータを用いてモデルの汎化性能を評価する
test_eval = best_model.evaluate(x_test, y_test)
print('Test Acc :', test_eval[1])